In [1]:
import torch
import torch_geometric
from torch_geometric.data import Data, DataLoader
import deepchem as dc
from rdkit import Chem
import pandas as pd
import numpy as np
import torch
from torch_geometric.data import Data, InMemoryDataset
import multiprocessing
   
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv , TransformerConv
from torch.optim.lr_scheduler import ReduceLROnPlateau

No normalization for SPS. Feature removed!
No normalization for AvgIpc. Feature removed!


Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead


Skipped loading modules with transformers dependency. No module named 'transformers'
cannot import name 'HuggingFaceModel' from 'deepchem.models.torch_models' (c:\Users\sarab\miniconda3\envs\ENV4\lib\site-packages\deepchem\models\torch_models\__init__.py)


In [2]:
from torch_geometric.data import InMemoryDataset, Data

class DeepChemToPyGDataset(InMemoryDataset):
    def __init__(self, data_list, transform=None):
        """
        Converts a list of PyG `Data` objects into an InMemoryDataset.
        
        Args:
            data_list (list of Data): List of PyG Data objects.
            transform (callable, optional): Optional transform applied to each data object.
        """
        super(DeepChemToPyGDataset, self).__init__('.', transform, None, None)
        self.data, self.slices = self.collate(data_list)

    def get(self, idx):
        """
        Retrieves the data object at the specified index.
        """
        return super().get(idx)


In [3]:
pyg_dataset = torch.load(r'C:\Users\sarab\OneDrive\Desktop\CSIR\generative_model_compare\Convertors\cleaned_frag_pyg_dataset.pth')
dataset=pyg_dataset

C:\Users\sarab\AppData\Local\Temp\ipykernel_20704\811436411.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pyg_dataset = torch.load(r'C:\Users\sarab\OneDrive\Desktop\CS

In [4]:
def load_data(file_path):
    pyg_dataset = torch.load(r'C:\Users\sarab\OneDrive\Desktop\CSIR\generative_model_compare\Convertors\cleaned_frag_pyg_dataset.pth')
    dataset_length = len(pyg_dataset)

    # 80% training, 20% validation split
    train_size = int(0.8 * dataset_length)
    val_size = dataset_length - train_size
    train_dataset, val_dataset = torch.utils.data.random_split(
        pyg_dataset, [train_size, val_size]
    )
    return train_dataset, val_dataset, pyg_dataset[0].num_features, pyg_dataset[0].edge_attr.size(-1)

In [5]:
node_features = pyg_dataset[0].x.shape[1]  # Should be 134 based on your description
edge_features = pyg_dataset[0].edge_attr.shape[1]  # Should be 6 based on your description


In [6]:
#new simple model
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool, global_max_pool

class ARGVA(nn.Module):
    def __init__(self, node_features, edge_features, hidden_dim, latent_dim):
        super(ARGVA, self).__init__()
        self.encoder = VariationalEncoder(node_features, edge_features, hidden_dim, latent_dim)
        self.decoder = Decoder(latent_dim, hidden_dim, node_features, edge_features)
        self.discriminator = Discriminator(latent_dim, hidden_dim)
        
    def encode(self, x, edge_index, edge_attr, batch=None):
        return self.encoder(x, edge_index, edge_attr, batch)

    def decode(self, z, edge_index, batch=None):
        return self.decoder(z, edge_index, batch)

    def discriminate(self, z):
        return self.discriminator(z)

    def reparameterize(self, mu, logvar):
        if self.training:
            std = torch.exp(0.5 * logvar)
            eps = torch.randn_like(std)
            return mu + eps * std
        return mu

    def forward(self, x, edge_index, edge_attr, batch=None):
        mu, logvar = self.encode(x, edge_index, edge_attr, batch)
        z = self.reparameterize(mu, logvar)
        reconstructed_x, reconstructed_edge_attr = self.decode(z, edge_index, batch)
        return reconstructed_x, reconstructed_edge_attr, mu, logvar

    def generate_multiple_outputs(self, x, edge_index, edge_attr, num_samples=5):
        outputs = []
        with torch.no_grad():
            mu, logvar = self.encode(x, edge_index, edge_attr)
            std = torch.exp(0.5 * logvar)
            
            for _ in range(num_samples):
                # Temperature annealing for diversity
                temperature = 1.0 / (1.0 + 0.1 * _)
                
                # Sample from latent space
                z = mu + temperature * std * torch.randn_like(std)
                
                # Decode
                reconstructed_x, reconstructed_edge_attr = self.decode(z, edge_index)
                
                # Apply temperature-scaled softmax
                reconstructed_x = torch.softmax(reconstructed_x / temperature, dim=-1)
                reconstructed_edge_attr = torch.softmax(reconstructed_edge_attr / temperature, dim=-1)
                
                outputs.append((reconstructed_x, reconstructed_edge_attr, edge_index))
        return outputs

class VariationalEncoder(nn.Module):
    def __init__(self, node_features, edge_features, hidden_dim, latent_dim):
        super(VariationalEncoder, self).__init__()
        
        # Multiple GCN layers with increasing expressiveness
        self.conv1 = GCNConv(node_features + edge_features, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim * 2)
        self.conv3 = GCNConv(hidden_dim * 2, hidden_dim)
        
        # Feature transformation
        self.transform = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.LeakyReLU(0.2)
        )
        
        # Latent space projections with more capacity
        self.mu = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.LeakyReLU(0.2),
            nn.Linear(hidden_dim, latent_dim)
        )
        
        self.logvar = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.LeakyReLU(0.2),
            nn.Linear(hidden_dim, latent_dim)
        )
        
    def forward(self, x, edge_index, edge_attr, batch=None):
        # Combine node features with edge features
        row, col = edge_index
        edge_attr_repeated = edge_attr.repeat(1, 1) if edge_attr.dim() == 3 else edge_attr
        
        # Aggregate edge features to nodes by averaging
        edge_feat_by_node = torch.zeros(x.size(0), edge_attr.size(1), device=x.device)
        for i in range(row.size(0)):
            edge_feat_by_node[row[i]] += edge_attr[i]
        
        # Normalize by counting occurrences of each node
        unique_nodes, counts = torch.unique(row, return_counts=True)
        edge_feat_by_node[unique_nodes] /= counts.float().unsqueeze(1)
        
        # Concatenate node features with aggregated edge features
        x = torch.cat([x, edge_feat_by_node], dim=-1)
        
        # Multi-layer graph convolutions with residual connections
        x1 = F.leaky_relu(self.conv1(x, edge_index))
        x2 = F.leaky_relu(self.conv2(x1, edge_index))
        x3 = F.leaky_relu(self.conv3(x2, edge_index))
        
        # Residual connection
        x = x3 + x1
        
        # Transform features
        x = self.transform(x)
        
        # Global pooling with both mean and max
        if batch is not None:
            x_mean = global_mean_pool(x, batch)
            x_max = global_max_pool(x, batch)
            x = (x_mean + x_max) / 2
        else:
            x = x.mean(dim=0, keepdim=True)
        
        # Project to latent space
        return self.mu(x), self.logvar(x)
class Decoder(nn.Module):
    def __init__(self, latent_dim, hidden_dim, node_features, edge_features):
        super(Decoder, self).__init__()
        
        # Increased network capacity with multiple hidden layers
        self.node_decoder = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim * 2),
            nn.LayerNorm(hidden_dim * 2),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.2),
            nn.Linear(hidden_dim * 2, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.1),
            nn.Linear(hidden_dim, node_features)
        )
        
        # Separate networks for different edge features to allow specialization
        self.edge_feature_nets = nn.ModuleList([
            nn.Sequential(
                nn.Linear(latent_dim * 2, hidden_dim),
                nn.LayerNorm(hidden_dim),
                nn.LeakyReLU(0.2),
                nn.Dropout(0.2),
                nn.Linear(hidden_dim, 1)
            ) for _ in range(edge_features)
        ])
        
        # Attention mechanism for edge features
        self.edge_attention = nn.Sequential(
            nn.Linear(latent_dim * 2, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.LeakyReLU(0.2),
            nn.Linear(hidden_dim, edge_features)
        )
        
        # Feature mixing layer
        self.feature_mixing = nn.Sequential(
            nn.Linear(edge_features, edge_features),
            nn.LayerNorm(edge_features),
            nn.LeakyReLU(0.2)
        )
        
        self.edge_features = edge_features
        
    def add_noise_to_latent(self, z, noise_factor=0.1):
        """Add small amount of noise to prevent mode collapse"""
        if self.training:
            noise = torch.randn_like(z) * noise_factor
            return z + noise
        return z
        
    def forward(self, z, edge_index, batch=None):
        # Add noise to latent representations during training
        z = self.add_noise_to_latent(z)
        
        # Handle batching
        if batch is not None:
            z = z[batch]
        else:
            z = z.repeat(edge_index.max().item() + 1, 1)
            
        # Reconstruct nodes
        reconstructed_x = self.node_decoder(z)
        
        # Edge reconstruction with attention and feature specialization
        row, col = edge_index
        edge_input = torch.cat([z[row], z[col]], dim=1)
        
        # Calculate attention weights for edge features
        attention_weights = F.softmax(self.edge_attention(edge_input), dim=-1)
        
        # Process each edge feature separately
        edge_outputs = []
        for i, net in enumerate(self.edge_feature_nets):
            feature_output = net(edge_input)
            weighted_output = feature_output * attention_weights[:, i].unsqueeze(1)
            edge_outputs.append(weighted_output)
        
        # Combine edge features
        edge_features = torch.cat(edge_outputs, dim=1)
        
        # Mix features to allow for feature interactions
        reconstructed_edge_attr = self.feature_mixing(edge_features)
        
        # Residual connection for edge features
        reconstructed_edge_attr = reconstructed_edge_attr + edge_features
        
        return reconstructed_x, reconstructed_edge_attr

class Discriminator(nn.Module):
    def __init__(self, latent_dim, hidden_dim):
        super(Discriminator, self).__init__()
        
        self.net = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1)
        )

    def forward(self, z):
        return torch.sigmoid(self.net(z))
# [Rest of the code remains exactly the same as before - Decoder and Discriminator classes are unchanged]

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Fixed hyperparameter
epochs = 60

In [8]:
import torch
import torch.nn.functional as F
import torch.optim as optim
from torch_geometric.loader import DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau

def train_argva_with_early_stopping(
    model, train_dataset, val_dataset, num_epochs, batch_size, lr, device, patience
):
    """
    Train the ARGVA model with early stopping based on validation loss.

    Parameters:
        model (nn.Module): ARGVA model instance
        train_dataset: Training dataset
        val_dataset: Validation dataset
        num_epochs (int): Maximum number of epochs for training
        batch_size (int): Batch size for training
        lr (float): Learning rate for the optimizer
        device (torch.device): Device to use for training ('cpu' or 'cuda')
        patience (int): Number of epochs to wait for improvement before stopping

    Returns:
        nn.Module: The best-trained model
    """
    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=patience//2, verbose=True)

    best_val_loss = float('inf')
    patience_counter = 0
    best_model_state = None

    for epoch in range(num_epochs):
        # Training phase
        model.train()
        total_loss = total_recon_loss = total_kl_loss = total_adv_loss = 0

        for batch in train_loader:
            batch = batch.to(device)
            optimizer.zero_grad()

            # Variational autoencoder forward pass
            reconstructed_x, reconstructed_edge_attr, mu, logvar = model(
                batch.x, batch.edge_index, batch.edge_attr, batch.batch
            )

            # Reconstruction loss
            recon_loss = F.mse_loss(reconstructed_x, batch.x) + \
                        F.mse_loss(reconstructed_edge_attr, batch.edge_attr)

            # KL divergence loss
            kl_loss = -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())

            # Adversarial loss
            z = model.reparameterize(mu, logvar)
            real_z = torch.randn_like(z)
            
            real_scores = model.discriminate(real_z)
            fake_scores = model.discriminate(z)
            
            adv_loss = -torch.mean(torch.log(real_scores + 1e-8) + torch.log(1 - fake_scores + 1e-8))
            

            

            # Total loss (with beta-VAE formulation)
            beta = 0.8  # Can be adjusted to control KL divergence impact # Orignal beta = 1
            loss = recon_loss + beta * kl_loss + 0.8 * adv_loss        #adv weight was 0.1

            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            total_recon_loss += recon_loss.item()
            total_kl_loss += kl_loss.item()
            total_adv_loss += adv_loss.item()

        avg_loss = total_loss / len(train_loader)
        avg_recon_loss = total_recon_loss / len(train_loader)
        avg_kl_loss = total_kl_loss / len(train_loader)
        avg_adv_loss = total_adv_loss / len(train_loader)

        # Validation phase
        model.eval()
        total_val_loss = 0
        
        with torch.no_grad():
            for batch in val_loader:
                batch = batch.to(device)
                reconstructed_x, reconstructed_edge_attr, mu, logvar = model(
                    batch.x, batch.edge_index, batch.edge_attr, batch.batch
                )

                recon_loss = F.mse_loss(reconstructed_x, batch.x) + \
                            F.mse_loss(reconstructed_edge_attr, batch.edge_attr)
                kl_loss = -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())
                
                val_loss = recon_loss + beta * kl_loss
                total_val_loss += val_loss.item()

        avg_val_loss = total_val_loss / len(val_loader)

        # Print progress
        print(
            f'Epoch [{epoch+1}/{num_epochs}] - '
            f'Loss: {avg_loss:.4f} '
            f'(Recon: {avg_recon_loss:.4f}, '
            f'KL: {avg_kl_loss:.4f}, '
            f'Adv: {avg_adv_loss:.4f}) - '
            f'Val Loss: {avg_val_loss:.4f}'
        )

        # Early stopping check
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            patience_counter = 0
            best_model_state = model.state_dict()
            print(f'New best model saved with validation loss: {best_val_loss:.4f}')
        else:
            patience_counter += 1
            print(f'No improvement. Patience counter: {patience_counter}/{patience}')

        if patience_counter >= patience:
            print('Early stopping triggered.')
            break

        scheduler.step(avg_val_loss)

    # Restore best model
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
        print('Best model state restored.')

    return model

In [9]:
import os
import json
import torch
import optuna
import multiprocessing
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch_geometric.loader import DataLoader as PyGDataLoader
from torch.utils.data import random_split

c:\Users\sarab\miniconda3\envs\ENV4\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
def objective(trial):
    hidden_dim = trial.suggest_int("hidden_dim", 64, 512, step=64)
    latent_dim = trial.suggest_int("latent_dim", 32, 128, step=32)
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True)
    batch_size = trial.suggest_categorical("batch_size", [8, 16, 32, 64])
    patience = trial.suggest_int("patience", 3, 10)

    # Create model and data loaders
    model = ARGVA(
        node_features=134,
        edge_features=6,
        hidden_dim=hidden_dim,
        latent_dim=latent_dim,
    ).to(device)
    print(f"\n[Trial {trial.number}] Starting with parameters: {trial.params}")
    train_ratio = 0.8
    pyg_dataset = torch.load(r'C:\Users\sarab\OneDrive\Desktop\CSIR\generative_model_compare\Convertors\cleaned_frag_pyg_dataset.pth')
    num_train = int(len(pyg_dataset) * train_ratio)
    num_valid = len(pyg_dataset) - num_train
    
    train_dataset, valid_dataset = random_split(pyg_dataset, [num_train, num_valid])
    
    # Create DataLoaders
    train_loader = PyGDataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = PyGDataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
    

    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    best_val_loss = float("inf")
    patience_counter = 0

    # Training loop
    for epoch in range(50):  # Set a max epoch limit
        model.train()
        total_loss = 0
        for batch in train_loader:
            batch = batch.to(device)
            optimizer.zero_grad()

            reconstructed_x, reconstructed_edge_attr, mu, logvar = model(
                batch.x, batch.edge_index, batch.edge_attr, batch.batch
            )

            # Loss calculation
            recon_loss = F.mse_loss(reconstructed_x, batch.x) + \
                        F.mse_loss(reconstructed_edge_attr, batch.edge_attr)
            kl_loss = -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())
            loss = recon_loss + 0.8 * kl_loss  # Adjust beta weight as needed

            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        # Validation phase
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for batch in val_loader:
                batch = batch.to(device)
                reconstructed_x, reconstructed_edge_attr, mu, logvar = model(
                    batch.x, batch.edge_index, batch.edge_attr, batch.batch
                )

                recon_loss = F.mse_loss(reconstructed_x, batch.x) + \
                            F.mse_loss(reconstructed_edge_attr, batch.edge_attr)
                kl_loss = -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())
                val_loss += (recon_loss + 0.8 * kl_loss).item()

        val_loss /= len(val_loader)
        print(f"Trial: {trial.number}, Epoch: {epoch}, Train Loss: {total_loss:.4f}, Val Loss: {val_loss:.4f}")

        # Early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= patience:
            print("Early stopping triggered.")
            break

    return best_val_loss



In [11]:
import os
if __name__ == "__main__":
    os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # Set GPU if available

    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=200, n_jobs=multiprocessing.cpu_count())

    print("Best Trial:")
    print(study.best_trial)

[I 2024-12-24 01:01:11,091] A new study created in memory with name: no-name-658b121d-34d7-4962-8c72-a44024240546



[Trial 12] Starting with parameters: {'hidden_dim': 64, 'latent_dim': 96, 'learning_rate': 1.8521640242697503e-05, 'batch_size': 16, 'patience': 5}

[Trial 15] Starting with parameters: {'hidden_dim': 320, 'latent_dim': 32, 'learning_rate': 0.0001271547083066484, 'batch_size': 32, 'patience': 7}

[Trial 13] Starting with parameters: {'hidden_dim': 192, 'latent_dim': 64, 'learning_rate': 2.4129905873466902e-05, 'batch_size': 64, 'patience': 6}

[Trial 11] Starting with parameters: {'hidden_dim': 256, 'latent_dim': 128, 'learning_rate': 0.0037979731323002277, 'batch_size': 64, 'patience': 7}

[Trial 4] Starting with parameters: {'hidden_dim': 512, 'latent_dim': 32, 'learning_rate': 0.012377766412990019, 'batch_size': 32, 'patience': 6}

[Trial 6] Starting with parameters: {'hidden_dim': 448, 'latent_dim': 128, 'learning_rate': 0.028291624277561673, 'batch_size': 64, 'patience': 8}

[Trial 14] Starting with parameters: {'hidden_dim': 448, 'latent_dim': 96, 'learning_rate': 0.034565586212

C:\Users\sarab\AppData\Local\Temp\ipykernel_20704\3486618961.py:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pyg_dataset = torch.load(r'C:\Users\sarab\OneDrive\Desktop\

Trial: 13, Epoch: 0, Train Loss: 53.9171, Val Loss: 0.5042
Trial: 11, Epoch: 0, Train Loss: 18.1106, Val Loss: 0.1384
Trial: 2, Epoch: 0, Train Loss: 13.0751, Val Loss: 0.2097
Trial: 9, Epoch: 0, Train Loss: 34.3219, Val Loss: 0.5527
Trial: 10, Epoch: 0, Train Loss: 16.4376, Val Loss: 0.2255
Trial: 6, Epoch: 0, Train Loss: 2669.1687, Val Loss: 2.8701
Trial: 5, Epoch: 0, Train Loss: 96.8008, Val Loss: 0.4497
Trial: 15, Epoch: 0, Train Loss: 31.7767, Val Loss: 0.2200
Trial: 4, Epoch: 0, Train Loss: 224.7597, Val Loss: 0.2469
Trial: 12, Epoch: 0, Train Loss: 214.4782, Val Loss: 0.5481
Trial: 0, Epoch: 0, Train Loss: 42.1634, Val Loss: 0.1634
Trial: 3, Epoch: 0, Train Loss: 104.8348, Val Loss: 0.9674
Trial: 1, Epoch: 0, Train Loss: 89.2637, Val Loss: 0.5089
Trial: 14, Epoch: 0, Train Loss: 24389.5296, Val Loss: 291.0067
Trial: 13, Epoch: 1, Train Loss: 30.7203, Val Loss: 0.6795
Trial: 11, Epoch: 1, Train Loss: 7.9594, Val Loss: 0.1257
Trial: 2, Epoch: 1, Train Loss: 8.2132, Val Loss: 0.233

[I 2024-12-24 01:18:53,139] Trial 5 finished with value: 0.4497102419535319 and parameters: {'hidden_dim': 128, 'latent_dim': 32, 'learning_rate': 1.527162761938593e-05, 'batch_size': 32, 'patience': 4}. Best is trial 5 with value: 0.4497102419535319.


...

Trial: 196, Epoch: 12, Train Loss: 50.6155, Val Loss: 0.1075
Early stopping triggered.
Trial: 190, Epoch: 17, Train Loss: 65.3269, Val Loss: 0.1315


[I 2024-12-24 13:25:53,746] Trial 195 finished with value: 0.10694021762920027 and parameters: {'hidden_dim': 64, 'latent_dim': 128, 'learning_rate': 0.0025710531079060975, 'batch_size': 8, 'patience': 5}. Best is trial 101 with value: 0.10151587203145027.


Trial: 195, Epoch: 13, Train Loss: 50.4288, Val Loss: 0.1123
Early stopping triggered.
Trial: 190, Epoch: 18, Train Loss: 64.6167, Val Loss: 0.1293
Trial: 190, Epoch: 19, Train Loss: 64.3469, Val Loss: 0.1283
Trial: 190, Epoch: 20, Train Loss: 63.0936, Val Loss: 0.1239
Trial: 190, Epoch: 21, Train Loss: 62.5866, Val Loss: 0.1234
Trial: 190, Epoch: 22, Train Loss: 62.2937, Val Loss: 0.1235
Trial: 190, Epoch: 23, Train Loss: 61.3581, Val Loss: 0.1229
Trial: 190, Epoch: 24, Train Loss: 60.6168, Val Loss: 0.1204
Trial: 190, Epoch: 25, Train Loss: 60.0044, Val Loss: 0.1208
Trial: 190, Epoch: 26, Train Loss: 59.2589, Val Loss: 0.1213
Trial: 190, Epoch: 27, Train Loss: 58.5816, Val Loss: 0.1228
Trial: 190, Epoch: 28, Train Loss: 58.1245, Val Loss: 0.1197
Trial: 190, Epoch: 29, Train Loss: 57.6206, Val Loss: 0.1237
Trial: 190, Epoch: 30, Train Loss: 57.0555, Val Loss: 0.1246
Trial: 190, Epoch: 31, Train Loss: 56.3912, Val Loss: 0.1266
Trial: 190, Epoch: 32, Train Loss: 55.9333, Val Loss: 0.130

[I 2024-12-24 13:28:36,938] Trial 190 finished with value: 0.11966487377130684 and parameters: {'hidden_dim': 64, 'latent_dim': 128, 'learning_rate': 3.334901367093877e-05, 'batch_size': 8, 'patience': 5}. Best is trial 101 with value: 0.10151587203145027.


Trial: 190, Epoch: 33, Train Loss: 55.5981, Val Loss: 0.1299
Early stopping triggered.
Best Trial:
FrozenTrial(number=101, state=TrialState.COMPLETE, values=[0.10151587203145027], datetime_start=datetime.datetime(2024, 12, 24, 8, 30, 41, 586558), datetime_complete=datetime.datetime(2024, 12, 24, 8, 59, 50, 470016), params={'hidden_dim': 64, 'latent_dim': 128, 'learning_rate': 0.0030668665466399994, 'batch_size': 16, 'patience': 4}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'hidden_dim': IntDistribution(high=512, log=False, low=64, step=64), 'latent_dim': IntDistribution(high=128, log=False, low=32, step=32), 'learning_rate': FloatDistribution(high=0.1, log=True, low=1e-05, step=None), 'batch_size': CategoricalDistribution(choices=(8, 16, 32, 64)), 'patience': IntDistribution(high=10, log=False, low=3, step=1)}, trial_id=101, value=None)
